In [44]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
import sys
sys.path.insert(0,"..")

# Import packages and setup base classes

In [46]:
import torch
import torch.optim as optim
from types import SimpleNamespace as Namespace

In [47]:
from utils.training_loop import TrainingLoop
from custom_datasets.messidor import Messidor
from utils.construct_model import LoadVICRegModel
from utils.training_loop import TrainingLoop


In [48]:
gpu = torch.cuda.current_device()
torch.cuda.set_device(gpu)
torch.backends.cudnn.benchmark = True

## namespace mock

In [49]:
args = Namespace(
    batch_size=16,
    num_workers=4,
    gpu=None,
    weight_decay=0.0001,
    epochs=20,
    weights="finetune",
    print_freq=50,
    exp_dir="./test_expr/",
    lr_backbone=1e-3,
    lr_head=0.03,
)


In [50]:
dataset = Messidor(
    batch_size=16,
    num_workers=4,
    gpu=None,
)

# Infra

## dataloaders

In [51]:
train_loader = dataset.get_dataloader(split="train")
val_loader = dataset.get_dataloader(split="valid")

length of dataset train: 1394
length of dataset valid: 350


## model

In [52]:
model = LoadVICRegModel(arch="resnet50")
model.load_pretrained_weights(pretrained_path="../VICReg_RadImageNet/resnet50.pth")
model.modify_head(num_classes=dataset.num_classes)
model = model.produce_model()
model.cuda(gpu)
param_groups = [dict(params=model[-1].parameters(), lr=args.lr_head)]
if args.weights == "finetune":
    param_groups.append(dict(params=model[:-1].parameters(), lr=args.lr_backbone))

Loading local VICReg ResNet50 arch Model


In [53]:
## optimizer
optimizer = optim.Adam(param_groups, 0, weight_decay=args.weight_decay)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, args.epochs)


# training loop


In [54]:
training_loop = TrainingLoop(
    model,
    optimizer,
    scheduler,
    train_loader,
    val_loader,
    dataset,
    args,
    gpu=gpu,
    stats_file=None,
    multi_label=False,
)

In [55]:
training_loop.train(
    start_epoch=0, num_epochs=args.epochs)

2025-01-21 07:09 [info     ] Beginning training            


2025-01-21 07:09 [info     ] {"epoch": 0, "step": 0, "lr_backbone": 0.001, "lr_head": 0.03, "loss": 1.5671162605285645, "time": 0}
2025-01-21 07:09 [info     ] {"epoch": 0, "step": 0, "lr_backbone": 0.001, "lr_head": 0.03, "loss": 1.5671162605285645, "time": 0}
2025-01-21 07:09 [info     ] {"epoch": 0, "step": 20, "lr_backbone": 0.001, "lr_head": 0.03, "loss": 1.0555644035339355, "time": 4}
2025-01-21 07:09 [info     ] {"epoch": 0, "step": 20, "lr_backbone": 0.001, "lr_head": 0.03, "loss": 1.0555644035339355, "time": 4}
2025-01-21 07:09 [info     ] {"epoch": 0, "step": 40, "lr_backbone": 0.001, "lr_head": 0.03, "loss": 0.6023654937744141, "time": 7}
2025-01-21 07:09 [info     ] {"epoch": 0, "step": 40, "lr_backbone": 0.001, "lr_head": 0.03, "loss": 0.6023654937744141, "time": 7}
2025-01-21 07:09 [info     ] {"epoch": 0, "step": 60, "lr_backbone": 0.001, "lr_head": 0.03, "loss": 1.2720930576324463, "time": 10}
2025-01-21 07:09 [info     ] {"epoch": 0, "step": 60, "lr_backbone": 0.001, "